### Readme

This notebook demonstrates how to preprocess human 10x Genomics data using Moana. The dataset used for this demonstration is called "[4k PBMCs from a Healthy Donor](https://support.10xgenomics.com/single-cell-gene-expression/datasets/2.1.0/pbmc4k)", and is part of a series of datasets provided by 10x Genomics as demonstrations of their "v2" chemistry.

This tutorial uses the following files that are included in the GitHub repository.
- `pbmc4k_filtered_gene_bc_matrices.tar.gz` ([10x scRNA-Seq expression matrix](http://cf.10xgenomics.com/samples/cell-exp/2.1.0/pbmc4k/pbmc4k_filtered_gene_bc_matrices.tar.gz))
- `Homo_sapiens.GRCh38.92.gtf.gz` ([Ensembl genome annotations](http://ftp.ensembl.org/pub/release-92/gtf/homo_sapiens/Homo_sapiens.GRCh38.92.gtf.gz))

In [1]:
# intialize logger
from moana import util
_LOGGER = util.get_logger()

### Preprocessing of the 10x Genomics data using Moana

In [2]:
import os
import sys

import pandas as pd
import numpy as np

from moana.core import ExpMatrix
from moana import ensembl
from moana import util
from moana import qc

seed = 0

expression_file = 'data/pbmc4k_filtered_gene_bc_matrices.tar.gz'
genome_annotation_file = 'data/Homo_sapiens.GRCh38.92.gtf.gz'

# get list of protein-coding genes, keep genes with duplicate Ensembl IDs
gene_table = ensembl.get_protein_coding_genes(genome_annotation_file, remove_duplicates=False)
print('Number of protein-coding Ensembl genes:', gene_table.shape[0])

# read 10x Genomics data, use Ensembl IDs
matrix = ExpMatrix.read_10xgenomics(expression_file, prefix='filtered_gene_bc_matrices/GRCh38/', use_ensembl_ids=True).astype(np.float64)
print('Full matrix:', matrix.shape)

# map Ensembl IDs back to gene names using the Ensembl annotations
matrix = util.convert_from_ensembl_ids(matrix, gene_table)
print('After mapping to protein-coding genes:', matrix.shape)

# remove genes encoding ribosomal proteins, and genes on the mitochondrial genome
matrix, qual = qc.filter_matrix(matrix, min_transcripts=0, max_mito_frac=1.0)
print('After filtering ribosomal and mitochondrial genes:', matrix.shape)

# shuffle the cells
matrix = matrix.sample(frac=1.0, axis=1, random_state=seed)

# output a hash that uniquely identifies this expression matrix
print('Final expression matrix hash:', matrix.hash)

# output expression matrix
output_file = 'output/pbmc4k_expression.mtx'
matrix.write_sparse(output_file)

# output some QC statistics for each cell
qual.write_tsv('output/pbmc4k_qc.tsv')

[2018-10-31 14:56:40] (moana.ensembl.annotations) INFO: Removed 0 protein-coding genes with source "ensembl" that also had manual annotations.
[2018-10-31 14:56:40] (moana.ensembl.annotations) INFO: Protein-coding genes with multiple Ensembl IDs:MATR3 (2), HSPA14 (2), POLR2J3 (2), SOD2 (2), DIABLO (2), ALDOA (2), COG8 (2), PRSS50 (2), IGF2 (2), EMG1 (2), FAM212B (2), TXNRD3NB (2), ATXN7 (2), TBCE (2), H2BFS (2), CCDC39 (2), SCO2 (2), PDE11A (2), TMSB15B (2), ABCF2 (2)
[2018-10-31 14:56:40] (moana.ensembl.annotations) INFO: Read 2689566 lines (in 269 chunks).
[2018-10-31 14:56:40] (moana.ensembl.annotations) INFO: Found 19950 valid gene entries.
[2018-10-31 14:56:40] (moana.ensembl.annotations) INFO: Final number of unique genes: 19950
[2018-10-31 14:56:40] (moana.ensembl.annotations) INFO: Parsing time: 22.3 s
[2018-10-31 14:56:40] (moana.ensembl.annotations) INFO: Valid chromosomes (40): 19, 10, 12, 1, 22, 3, 4, 9, 15, 2, 11, 17, 20, 8, 16, 6, 7, X, 13, 14, 21, 18, 5, KI270721.1, KI27